<a href="https://colab.research.google.com/github/pcbzmani/Learning-repository-for-scala-and-pyspark/blob/main/Solutions/Python/date_range.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 63.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=f293192d45f0c54135c5566e75dd0ca8dcdbc4f6b63493d8cddc7fe7c8064854
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("daterange")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [4]:
data_rnge = [('2021-01-01','2021-01-31'),('2021-02-01','2021-02-28'),('2021-03-01','2021-03-26')]
cols_dtr = ['start_dt','end_dt']

In [14]:
rdd = spark.sparkContext.parallelize(data_rnge)
date_rng_df = rdd.toDF(cols_dtr)

In [15]:
date_rng_df.show()

+----------+----------+
|  start_dt|    end_dt|
+----------+----------+
|2021-01-01|2021-01-31|
|2021-02-01|2021-02-28|
|2021-03-01|2021-03-26|
+----------+----------+



In [8]:
each_day=[('2021-01-01',30),('2021-01-02',45),('2021-01-29',100),('2021-02-01',890),('2021-02-22',895),('2021-03-01',100)]
cols_each_day = ['date_sold','amount_sold']

In [10]:
rdd1 = spark.sparkContext.parallelize(each_day)
date_sold_df = rdd1.toDF(cols_each_day)

In [13]:
date_sold_df.show()

+----------+-----------+
| date_sold|amount_sold|
+----------+-----------+
|2021-01-01|         30|
|2021-01-02|         45|
|2021-01-29|        100|
|2021-02-01|        890|
|2021-02-22|        895|
|2021-03-01|        100|
+----------+-----------+



In [17]:
join_cond = [date_sold_df.date_sold >=date_rng_df.start_dt,date_sold_df.date_sold <= date_rng_df.end_dt]
join_type = 'inner'

In [18]:
joined_df = date_sold_df.join(date_rng_df,join_cond,join_type)

In [19]:
joined_df.show()

+----------+-----------+----------+----------+
| date_sold|amount_sold|  start_dt|    end_dt|
+----------+-----------+----------+----------+
|2021-01-01|         30|2021-01-01|2021-01-31|
|2021-01-02|         45|2021-01-01|2021-01-31|
|2021-01-29|        100|2021-01-01|2021-01-31|
|2021-02-01|        890|2021-02-01|2021-02-28|
|2021-02-22|        895|2021-02-01|2021-02-28|
|2021-03-01|        100|2021-03-01|2021-03-26|
+----------+-----------+----------+----------+



In [ ]:
from pyspark.sql.functions import sum

In [21]:
result_df = joined_df.groupBy('start_dt','end_dt')\
.sum('amount_sold').alias('total_amount')

In [22]:
result_df.show()

+----------+----------+----------------+
|  start_dt|    end_dt|sum(amount_sold)|
+----------+----------+----------------+
|2021-03-01|2021-03-26|             100|
|2021-01-01|2021-01-31|             175|
|2021-02-01|2021-02-28|            1785|
+----------+----------+----------------+

